In [1]:
import pandas as pd

In [7]:
my_answer =  pd.read_csv('../data/first_answer.csv', index_col=0)
my_answer


,0
0,"://사고사례 전파 및 안전교육 실시, 현장 시공사례 전파 및 현장 안전교육 실시,..."
1,"Question: 사고사례 전파 및 재발 방지 대책 수립, 안전교육 강화, 안전보건..."
2,"Questionnaire 작성 및 사고 사례 전파, 근로자 교육 실시, 위험요소 제..."
3,Question: 작업발판을 사용하여 작업을 실시하는 작업자 안전교육 및 작업발판 ...
4,"defence, 지하공간 내 안전교육 실시와 작업장 내 안전교육 강화, 작업장 내 ..."
...,...
959,"A형근골절 재발 방지 및 재발 방지 대책 수립, 근골절 사고 사례 전파 및 재발 방..."
960,The work area is planned to be secured by expa...
961,"1) 작업자 안전교육 실시 및 안전관리 철저, 2) 작업자 안전교육 실시 및 안전관..."
962,"Isolating the area, cleaning up debris, and co..."


In [8]:
sub_samp =  pd.read_csv('../data/sample_submission.csv')

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = list(my_answer.iloc[:, 0])

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sbert-sts')
model = AutoModel.from_pretrained('jhgan/ko-sbert-sts')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])


In [ ]:
tensor_df =  pd.DataFrame(sentence_embeddings.numpy())
sample_parsed =  pd.concat( [sub_samp.iloc[:, :2], sentence_embeddings], axis = 1 )

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.717093,0.495281,-1.083745,0.080659,0.354890,0.519397,-0.071754,0.283470,0.278073,-0.011440,...,0.805345,0.949291,-0.122635,1.408146,0.608274,0.329519,0.292418,0.644906,0.364845,-0.383166
1,0.023746,0.464125,-0.239515,-0.161702,0.133671,0.010160,0.238808,-0.329044,0.357545,-0.271792,...,0.287229,1.589413,0.430770,1.906507,0.248541,-0.590214,-0.002421,0.162994,0.525059,0.153331
2,-0.216507,0.942671,-0.122733,0.379575,0.338922,0.927729,-0.182440,-0.567794,0.564505,-0.212212,...,0.265206,1.181859,0.186401,1.504728,0.502603,-0.679978,-0.225414,0.092848,0.755934,0.179091
3,0.388258,0.813992,-0.520967,0.041387,-0.256227,0.803138,0.666095,-0.152065,0.148048,0.072249,...,-0.232333,0.806007,-0.339313,0.920434,0.190048,-0.167593,0.000723,0.266885,0.234810,0.073036
4,-0.051078,0.143457,-0.572787,-0.015971,-0.383871,0.365102,-0.059611,0.256686,0.597952,0.824465,...,0.157830,1.728342,0.770699,0.641549,0.216704,0.054555,0.462741,-0.101585,0.690635,0.030269
5,-0.331526,-0.056471,-0.418523,0.933338,-0.562712,-1.484085,-0.539346,-0.655537,0.325738,-0.515303,...,0.620691,0.689603,0.879140,-0.902889,-1.097203,-0.362964,1.319030,-0.197952,0.183079,0.909861
6,-0.061532,0.294382,-0.078753,0.014995,0.482047,-0.069403,0.371460,-0.378521,0.184785,0.557968,...,0.948591,1.238093,1.295091,2.104946,0.709421,-0.953280,0.303746,-0.099187,1.083843,-0.387359
7,0.664590,0.857388,-0.381713,0.515419,0.052855,-0.237591,0.760149,-0.045972,0.900498,-0.338208,...,0.603520,0.086878,0.413875,-0.438497,-0.871438,-0.984050,0.179029,-0.182113,-0.461268,0.737449
8,-0.712375,0.805003,-0.242228,1.070915,-0.531542,0.399064,1.339332,-0.421070,0.011445,-0.050444,...,0.660567,1.507848,-0.256573,1.128463,0.572518,-0.122907,-0.075461,0.018038,0.011048,0.072996
9,-0.514480,0.527419,-0.204060,0.728771,-0.484294,0.207565,0.892141,0.068679,-0.229052,0.315055,...,0.287349,0.290355,-0.388667,0.902804,0.622462,-0.110996,-0.317732,-0.366488,0.464853,-0.513942
